In [1]:
import pdfplumber, re,  sys, time, pandas, json, glob

In [2]:
Header_Data0={"Workfile":"","PublishedDate":"","PublishedYear":"","AssignmentDate":"","Customer":"","CustomerAddress":"","Client":"","MarketArea":""}
Asset_Data0={"AssetTitle":"","AssetYear":"","AssetType":"","AssetMake":"","AssetModel":"","SerialNumber":"","FMV_Low":"","FMV_High":"","OLV_Low":"","OLV_High":""}
Asset_Specs0={"Kms":"","Hours":"","Miles":"","Engine":"","EnginePower":"","Transmission":"","AxlesNo":"",
              "GVWR":"","RearAxle":"","FrontAxle":"","Sleeper":"","MooseBumper":""}
Asset_Dscp0={"Mechanical":"","Exterior":"","Interior":"","Type":"","Condition":""}
Market_Rsrch0={"Ad1":"","Ad2":"","Ad3":"","Ad4":"","Auction1":"","Auction2":"","Auction3":"","Dealer1":"","Dealer2":"","Dealer3":""}
Other_Specs0={}
for spec in range(1,40):
        Other_Specs0['Spec'+str(spec)] =""
        
JsonKeys={**Header_Data0,**Asset_Data0,**Asset_Specs0,**Asset_Dscp0,**Market_Rsrch0,**Other_Specs0}


header_data={}
asset_data={}
asset_specs={}
json_data = {}
market_research={}
other_specs={}
asset_dscp={}

header_data.update(Header_Data0)
asset_data.update(Asset_Data0)
asset_specs.update(Asset_Specs0)
json_data = JsonKeys
market_research.update(Market_Rsrch0)
other_specs.update(Other_Specs0)
asset_dscp.update(Asset_Dscp0)

In [3]:
#1&2
def readfiles():
    pdfs = []
    for file in glob.glob("*.pdf"):
        pdfs.append(file)
    print(pdfs)
    return pdfs


# Preprocesses the raw text by removing all new lines and returning a dict (seperated by what was a newline)
def preprocess_data(raw_text):
    return [el.strip() for el in raw_text.splitlines() if el.strip()]


# Get data that is always in the same spot.
def get_headerData(raw_text, header_data = {}):
    Worked=1
    try:
        workfile = re.search(r'(\D{5}\d{10})',raw_text).group(1).strip()
        header_data['Workfile'] = workfile
        #processed_text = preprocess_data(raw_text)
        # Data that is always in the same spot
    except:
        Worked=0
        return Worked
        print("Workfile Name ERROR")
    if Worked==1:
        try:
            header_data['PublishedDate']=re.search(r'Publication Date:\s*(.{1,})\n',raw_text).group(1).strip()
            header_data['PublishedYear']=re.search(r'Publication Date:\s*(\w{3,12}\s+\d{1,2}\W\s)(\d{4})\n',raw_text).group(2).strip()

        except:
            try:
                header_data['PublishedDate'] = re.search(r'(\w{3,12}\s+\d{1,2}\W\s\d{4})',raw_text).group(1).strip()
                header_data['PublishedYear'] = re.search(r'(\w{3,12}\s+\d{1,2}\W\s)(\d{4})',raw_text).group(2).strip()

            except:
                print("header_data['Published Date']-- ERROR")
                pass

        try:
            header_data['Client'] = re.search(r'(Attention)(\s{1,})(\w{1,}\s{1,}\w{1,})',raw_text).group(3).strip()

        except:
            print("header_data['Client']-- ERROR")
            pass
        try:
            header_data['Customer'] = re.search(r'\n(.*\s*)(.*\s*.*\w\d\w\s{0,1}\d\w\d\s*)(Attention)',raw_text).group(1).strip()

        except:
            print("header_data['Customer']-- ERROR")
            pass
        try:
            header_data['CustomerAddress'] = re.search(r'\n(.*\s*)(.*\s*.*\w\d\w\s{0,1}\d\w\d\s*)(Attention)',raw_text).group(2).strip()

        except:
            print("header_data['CustomerAddress']-- ERROR")
            pass

        
        try:
            header_data['AssignmentDate'] = re.search(r'(Assignment.*:\s*)(.*)',raw_text).group(2).strip()

        except:
            print("header_data['AssignmentDate']-- ERROR")
            pass

        try:
            header_data['MarketArea'] = re.search(r'(Market Area\s{1,}:\s*)(.{1,})(\s{1})',raw_text).group(2).strip()

        except:
            print("header_data['MarketArea']-- ERROR")
            pass
        return Worked

In [17]:
#3&4
def get_assetData(raw_text, asset_data):
    KK2=1
    Nline_raw_text=re.sub("\n"," ",raw_text)
    Nline_raw_text=re.sub("  "," ",Nline_raw_text)
    Nline_raw_text=re.sub("  "," ",Nline_raw_text)
    Nline_raw_text=re.sub("  "," ",Nline_raw_text)
    
    try:
        asset_data['AssetTitle'] = re.search(r'described (.{1,}) based on',Nline_raw_text).group(1).strip()
    except AttributeError:
        try:
            asset_data['AssetTitle'] = re.search(r'(Item\s{1,}:\s{1,})(\d{4}?.{1,})(Serial Number)',Nline_raw_text).group(2).strip()
        except:
            pass

    try:
        asset_data['AssetYear'] = re.search(r'^\s*(\d{4})',asset_data['AssetTitle']).group(1).strip()
    except:
        pass 
    
    try:
        asset_data['AssetMake'] = re.search(r'^\s*(\d{4})\s(\w+)',asset_data['AssetTitle']).group(2).strip()
    except:
        pass
    
    try:
        asset_data['AssetType'] = re.search(r'(\w*\s\w*)\s*$',asset_data['AssetTitle']).group(1).strip()
    except:
        pass 
    
    try:
        asset_data['AssetModel'] = re.search(r'^\s*(\d{4})\s(\w+)(.*)',asset_data['AssetTitle']).group(3).strip()
    except:
        pass 
    
    try:
        asset_data['SerialNumber'] = re.search(r'(Serial Number\s{1,}:\s*)(.{6,17})',Nline_raw_text).group(2).strip()
    except :
        try:
            asset_data['SerialNumber'] = re.search(r'(VIN\s*)(.{17})',Nline_raw_text).group(2).strip()
        except:
            pass
    
    try:# FMV/ACV FMV high low/single
        asset_data['FMV_Low'] = re.search(r'((Fair Market Value|(Actual Cash Value)) (of|in the)\s{1,}\$)((\d{1,}\,)?(\d{1,}))',Nline_raw_text).group(5).strip()
        asset_data['FMV_High'] = re.search(r'((Fair Market Value|(Actual Cash Value)) (of|in the)\s{1,}\$)((\d{1,}\,)?(\d{1,}))(\s*-\s*\$*)((\d{1,}\,)?(\d{1,}))',Nline_raw_text).group(9).strip()
    except :
        try:
            asset_data['FMV_Low'] = re.search(r'((Fair Market Value|(Actual Cash Value)) (?:\(removed\)) (of|in the)\s{1,}\$)((\d{1,}\,)?(\d{1,}))',Nline_raw_text).group(5).strip()
            asset_data['FMV_High'] = re.search(r'((Fair Market Value|(Actual Cash Value)) (?:\(removed\)) (of|in the)\s{1,}\$)((\d{1,}\,)?(\d{1,}))(\s*-\s*\$*)((\d{1,}\,)?(\d{1,}))',Nline_raw_text).group(9).strip()
        except:
            pass


    
    try:# OLV/Auction value low/single
        asset_data['OLV_Low'] = re.search(r'(auction\s*\w* (of|in the)\s{1,}\$)(\d{1,}(\,\d{2,})?)((\s*.{1}\s*\$*)(\d{1,}(\,\d{2,})?))',Nline_raw_text).group(3).strip()
        asset_data['OLV_High'] = re.search(r'(auction\s*\w* (of|in the)\s{1,}\$)(\d{1,}(\,\d{2,})?)((\s*.{1}\s*\$*)(\d{1,}(\,\d{2,})?))',Nline_raw_text).group(7).strip()
    except AttributeError:
        try:
            asset_data['OLV_Low'] = re.search(r'((Orderly Liquidation Value) (of|in the)\s{1,}\$)((\d{1,}\,)?(\d{1,}))',Nline_raw_text).group(4).strip()
            asset_data['OLV_High'] = re.search(r'((Orderly Liquidation Value) (of|in the)\s{1,}\$)((\d{1,}\,)?(\d{1,}))(\s*-\s*\$*)((\d{1,}\,)?(\d{1,}))',Nline_raw_text).group(8).strip()
        except:
            try:
                asset_data['OLV_Low'] = re.search(r'((Orderly and Forced Liquidation Value) (of|in the)\s{1,}\$)((\d{1,}\,)?(\d{1,}))',Nline_raw_text).group(4).strip()
                asset_data['OLV_High'] = re.search(r'((Orderly and Forced Liquidation Value) (of|in the)\s{1,}\$)((\d{1,}\,)?(\d{1,}))(\s*-\s*\$*)((\d{1,}\,)?(\d{1,}))',Nline_raw_text).group(8).strip()
            except:

                pass

    if len(re.findall(r'and',asset_data['AssetTitle']))>0:
        print("E1")
        KK2=0
        
    if len(re.findall(r'/',asset_data['AssetTitle']))>0:
        print("E1")
        KK2=0
    if len(re.findall(r'auction',Nline_raw_text))>2:
        KK2=0
        print("E2")

    if len(re.findall(r'Fair Market Value',Nline_raw_text))>2:
        KK2=0
        print("E3")

    return KK2

######__________-------------------------__________-------------------------__________-------------------------

def get_assetSpecs(raw_text, asset_specs):
    raw_text=raw_text.lower()
    try:
        asset_specs['Kms'] = re.search(r'(?:\•\s{4,8})(.*)(km|kms|kilometers)',raw_text).group(1).strip()
    except:
        pass
    
    try:
        asset_specs['EnginePower'] = re.search(r'(?:\•\s{4,8})(?:.*)(\d{3}\s{0,1})(?:hp)',raw_text).group(1)
    except:
        pass
    
    try:
        asset_specs['Miles'] = re.search(r'(?:\•\s{4,8})(.*)(miles|mile)',raw_text).group(1).strip()
    except:
        pass
    
    try:
        asset_specs['Hours'] = re.search(r'(?:\•\s{4,8})(?:.*)(.{8})\s*(?:(hours))',raw_text).group(1).strip()
    except:
        try:
            asset_specs['Hours'] = re.search(r'(?:\•\s{4,8})(.*)(?:(hours))',raw_text).group(1).strip()
        except:
            pass    
       
    try:
        asset_specs['Engine'] = re.search(r'(?:\•\s{4,8})(.*engine)',raw_text).group(1)
    except:
        try:
            asset_specs['Engine'] = re.search(r'(?:\•\s{4,8})(.*diesel)',raw_text).group(1)
        except:
            pass
    try:
        asset_specs['Transmission'] = re.search(r'(?:\•\s{4,8})(.*)(transmission)',raw_text).group(1)
    except:
        pass
    
    try:
        asset_specs['AxlesNo'] = re.search(r'(tandem|tridem|tri|(single axles{0,1})|(dual axles{0,1}))',raw_text).group(1)
    except:
        pass
    
    try:
        asset_specs['GVWR'] = re.search(r'(?:\•\s{4,8})(.*)(gvwr)',raw_text).group(1)
    except:
        pass
    
    try:
        asset_specs['RearAxle'] = re.search(r'(?:\•\s{4,8})(.*)(rear axles{0,1})',raw_text).group(1)
    except:
        pass

    try:
        asset_specs['FrontAxle'] = re.search(r'(?:\•\s{4,8})(.*)(front axle)',raw_text).group(1)
    except:
        pass

    try:
        asset_specs['Sleeper'] = re.search(r'(?:\•\s{4,8})(.*sleeper)',raw_text).group(1)
    except:
        pass
    
    try:
        asset_specs['MooseBumper'] = re.search(r'(?:\•\s{4,8})(.*)(moose|deer)',raw_text).group(1)
    except:
        pass


In [18]:
#5&6
def get_OtherSpecs(raw_text, other_specs):
    Nline_raw_text=re.sub('\•','\n•',raw_text)
    Nline_raw_text=re.sub('\n\n','\n',Nline_raw_text)

    raw_specs = re.findall(r'(?:\•\s{4,8})(.{1,})(?:\n)?',Nline_raw_text)

    index = 0
    for spec in raw_specs:
        #spec=re.sub(r"\&"," ",spec).strip()
        other_specs['Spec'+str(index+1)] = raw_specs[index].strip()
        index += 1

        
        
#6
def get_assetDscp(raw_text, asset_dscp):
    try:
        asset_dscp["Condition"] = re.search(r'(good|bad|fair|poor)(?:.{0,10})(?:condition)',raw_text).group(1)
    except:
        pass

    try:
        asset_dscp['Type'] = re.search(r'((highway tractor)|truck|RV|unit)',raw_text).group(1).strip()
    except:
        pass

    try:
        asset_dscp['Mechanical'] = re.search(r'((?:Mechanical Description).*(good|bad|fair|poor).*condition',raw_text).group(1).strip()
    except:
        pass

    try:
        asset_dscp['Exterior'] = re.search(r'(?:Exterior).*(good|bad|fair|poor).*condition',raw_text).group(1).strip()
    except:
        pass


    try:
        asset_dscp['Interior'] = re.search(r'(?:Interior).*(good|bad|fair|poor).*condition',raw_text).group(1).strip()
    except:
        pass







In [19]:
argv = readfiles()

['ACCRD2018020001.1pdf.pdf', 'ACCRD2018020001.pdf', 'ACCRD2018040001.pdf', 'ACCRD2018040002.pdf', 'ACCRD2018040003.pdf', 'ACCRD2018070001.pdf', 'ACCRD2018070002.pdf', 'ACCRD2018080001.pdf', 'ACCRD2018080002.pdf', 'ACCRD2018080003.pdf', 'ACCRD2018080004.pdf', 'ACCRD2018080005.pdf', 'ACCRD2018080006.pdf', 'ACCRD2018080007.pdf', 'ACCRD2018090001.1.pdf', 'ACCRD2018090001.pdf', 'ACCRD2018110001.pdf', 'ACCRD2018120013.pdf', 'ACCRD2018120014.pdf', 'ACCRD2018120015.pdf', 'ACCRD2018120016.pdf', 'ADEDM2018010001.pdf', 'ADEDM2018010002.pdf', 'ADEDM2018010003.pdf', 'ADEDM2018010004.pdf', 'ADEDM2018010005.pdf', 'ADEDM2018010006.pdf', 'ADEDM2018010007.pdf', 'ADEDM2018010008.pdf', 'ADEDM2018010009.pdf', 'ADEDM2018010010 (1).pdf', 'ADEDM2018010010.pdf', 'ADEDM2018010011.pdf', 'ADEDM2018010012.pdf', 'ADEDM2018010013.pdf', 'ADEDM2018010015.pdf', 'ADEDM2018010017.pdf', 'ADEDM2018010019.pdf', 'ADEDM2018020001.pdf', 'ADEDM2018020002.pdf', 'ADEDM2018020003.pdf', 'ADEDM2018020004.pdf', 'ADEDM2018020005.pdf',

In [21]:
#Main Script



print("Total Files: "+ str(len(argv)))

#JsonKeys={**Header_Data,**Asset_Data,**Asset_Specs,**Asset_Dscp,**Market_Rsrch,**Other_specs}


workfile = ""

header_data={}
asset_data={}
asset_specs={}
json_data = {}
market_research={}
other_specs={}
asset_dscp={}

header_data.update(Header_Data0)
asset_data.update(Asset_Data0)
asset_specs.update(Asset_Specs0)
json_data = JsonKeys
market_research.update(Market_Rsrch0)
other_specs.update(Other_Specs0)
asset_dscp.update(Asset_Dscp0)

curr_time = time.strftime("%H%M%S %Y%m%d")
#json_data = {**header_data, **asset_data, **asset_specs}
# Run for each pdf given as an argument
iterator=0
iter2=0
pdf_name_length_Error=[]
pdf_not_typical_version=[]
pdf_not_typical_2PlusAssets=[]

for file in argv:
    header_data.update(Header_Data0)
    asset_data.update(Asset_Data0)
    asset_specs.update(Asset_Specs0)
    json_data = JsonKeys
    market_research.update(Market_Rsrch0)
    other_specs.update(Other_Specs0)
    asset_dscp.update(Asset_Dscp0)
    Whole_pdf_raw=""
    iterator+= 1
    #print("\n"+"OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO ")
    print("PDF No: "+str(iterator)+"---------------START--------------- ") 
    if len(file)==19:
        try:
            with pdfplumber.open(file) as pdf:
                index = 0
                raw_text = ''
                # Extract all raw text
                #print("PDF PAGES:-----"+str(len(pdf.pages)))
                RunPdf=1
                while(index < len(pdf.pages)):
                    if RunPdf!=1:
                        index=10000
                    else:
                        page = pdf.pages[index]
                        raw_text = '\n' + page.extract_text()
                        Whole_pdf_raw=Whole_pdf_raw+raw_text
                        if index ==0 :
                            KK1=get_headerData(raw_text, header_data)
                            KK2=get_assetData(raw_text, asset_data)
                        if KK1==0:
                            pdf_not_typical_version.append(file)
                            #print("PDF No: "+str(iterator)+"---------------XX Not Typical PDF Version XX--------------- ")
                            #print("Not Typical PDF: "+file)
                            RunPdf=0
                        if KK2==0:
                            pdf_not_typical_2PlusAssets.append(file)
                            #print("PDF No: "+str(iterator)+"---------------XX Not Typical PDF 2+ Assets XX--------------- ")
                            #print("NNot Typical PDF 2+ Assets: "+file)
                            RunPdf=0
                        if KK1==0 or KK2==0:
                            continue
                        
                            
                        if index==1 and RunPdf!=0:  
                            
                            get_OtherSpecs(raw_text, other_specs)
                            get_assetSpecs(raw_text, asset_specs)

                        #if index==2 and RunPdf!=0:
                            
                            #get_assetDscp(raw_text, asset_dscp)
                            #get_Market_Rsrch(raw_text, asset_specs)
                            #get_OtherSpecs(raw_text, asset_specs)

                            
                        index +=1
                if RunPdf!=0:
                    #print(file)
                    #print(asset_data)
                    iter2+=1
                    json_data = {**header_data, **asset_data, **asset_specs,**asset_dscp, **other_specs}
                    #print(json_data)       
                    df = pandas.DataFrame(json_data, index=[iter2])
                    if iter2==1:
                        df.to_csv('test {}.csv'.format(curr_time), mode='a', header=True)
                    else :
                        df.to_csv('test {}.csv'.format(curr_time), mode='a', header=False)
                    header_data.update(Header_Data0)
                    asset_data.update(Asset_Data0)
                    asset_specs.update(Asset_Specs0)
                    json_data = JsonKeys
                    market_research.update(Market_Rsrch0)
                    other_specs.update(Other_Specs0)
                    asset_dscp.update(Asset_Dscp0)
                    #print(str(len(Whole_pdf_raw)))
           
            #print("PDF No: "+str(iterator)+"---------------FINISH--------------- ")               
        except OSError:
            print("For Loop ERROR:"+file)

            
    else:
        pdf_name_length_Error.append(file)
        #print("PDF No: "+str(iterator)+"---------------XX File Name Length Error XX--------------- ") 
    
     
#log.close()
print("\n\n")
print("pdfs_not_worked_NameLength: ",str(len(pdf_name_length_Error)))
print("pdf_not_typical_version: ",str(len(pdf_not_typical_version)))
print("Not Typical PDF 2+ Assets: ",str(len(pdf_not_typical_2PlusAssets)))




Total Files: 957
PDF No: 1---------------START--------------- 
PDF No: 2---------------START--------------- 
E1
PDF No: 3---------------START--------------- 
PDF No: 4---------------START--------------- 
PDF No: 5---------------START--------------- 
PDF No: 6---------------START--------------- 
PDF No: 7---------------START--------------- 
PDF No: 8---------------START--------------- 
PDF No: 9---------------START--------------- 
E1
PDF No: 10---------------START--------------- 
PDF No: 11---------------START--------------- 
PDF No: 12---------------START--------------- 
PDF No: 13---------------START--------------- 
PDF No: 14---------------START--------------- 
PDF No: 15---------------START--------------- 
PDF No: 16---------------START--------------- 
PDF No: 17---------------START--------------- 
PDF No: 18---------------START--------------- 
PDF No: 19---------------START--------------- 
PDF No: 20---------------START--------------- 
PDF No: 21---------------START--------------- 

AttributeError: 'NoneType' object has no attribute 'group'

In [ ]:
# Main Function
workfile = ""
# Setup CSV
#curr_time = time.strftime("%Y%m%d-%H%M%S")
#log = open(str(curr_time) + ".log","w")

def main():
    # Check if we are running on all pdfs
    print("1")
    argv = readfiles()
    asset_data = {}
    asset_specs = {}
    json_data = {}
    # Run for each pdf given as an argument
    iterator=0
    for file in argv:
        iterator= iterator+1
        print("PDF No:---------------START--------------- "+str(iterator)) 
        try:
            header_data = {}
            with pdfplumber.open(file) as pdf:
                index = 0
                raw_text = ''
                # Extract all raw text
                print("PDF PAGES:-----"+str(len(pdf.pages)))
                while(index < len(pdf.pages)):
                    page = pdf.pages[index]
                    raw_text = '\n' + page.extract_text()
                    if index ==0 :
                        get_headerData(raw_text, header_data)

                    index += 1
            print("PDF No:---------------FINISH--------------- "+str(iterator))            
        except OSError:
            print("ERROR")
    #log.close()
    print("1")

if (__name__ == '__main__'):
    main()
    

In [274]:
Description_data={}
with pdfplumber.open('ADECG2010120001.pdf') as pdf:
    try:
        page = pdf.pages[2]
        raw_text = '\n' + page.extract_text()
        raw_text=re.sub("\n"," ",raw_text)
        Description_data['Type'] = re.search(r'(truck|RV)',raw_text).group(1).strip()
        #Description_data['Exterior'] = re.search(r'(Exterior',raw_text).group(1).strip()
        #Description_data['Interior'] = re.search(r'(Interior',raw_text).group(1).strip()

        Description_data['Mechanical'] = re.search(r'(Mechanical Description).*(good|bad|fair|poor).*condition',raw_text).group(2).strip()
    except AttributeError:
        print("Not")
        pass
raw_text
Description_data

FileNotFoundError: [Errno 2] No such file or directory: 'ADECG2010120001.pdf'

In [275]:
#Needs debugging Description and Ads
Description_data={}
with pdfplumber.open('ADECG2010120001.pdf') as pdf:
    try:
        page = pdf.pages[2]
        raw_text = '\n' + page.extract_text()
        raw_text
        Description_data['Type'] = re.search(r'(truck|RV)',raw_text).group(1).strip()
        Description_data['Exterior'] = re.search(r'(Exterior',raw_text).group(1).strip()
        Description_data['Interior'] = re.search(r'(Interior',raw_text).group(1).strip()

        Description_data['Mechanical'] = re.search(r'(Mechanical Description ',raw_text).group(1).strip()
    except AttributeError:
        print("Not")
        pass
raw_text
Description_data



Ad_data={}
with pdfplumber.open('ADECG2010120001.pdf') as pdf:
    try:
        page = pdf.pages[5]
        raw_text = '\n' + page.extract_text()
        raw_text

        Description_data['Ads'] = re.search(r'(Market Research',raw_text).group(1).strip()
        Description_data['Auctions'] = re.search(r'(Clsoing',raw_text).group(1).strip()

    except AttributeError:
        print("Not")
        pass
raw_text
ad_data

FileNotFoundError: [Errno 2] No such file or directory: 'ADECG2010120001.pdf'

In [276]:
asset_data.update(Asset_Data0)
with pdfplumber.open('ADESK2020110002.pdf') as pdf:   
    page = pdf.pages[0]
    raw_text = '\n' + page.extract_text()
    Nline_raw_text=re.sub("\n"," ",raw_text)
    Nline_raw_text=re.sub("  "," ",Nline_raw_text)
    Nline_raw_text=re.sub("  "," ",Nline_raw_text)
    Nline_raw_text=re.sub("  "," ",Nline_raw_text)
    
    try:
        asset_data['AssetTitle'] = re.search(r'described (.{1,}) based on',Nline_raw_text).group(1).strip()
    except AttributeError:
        try:
            asset_data['AssetTitle'] = re.search(r'(Item\s{1,}:\s{1,})(\d{4}?.{1,})(Serial Number)',Nline_raw_text).group(2).strip()
        except:
            pass
        
    try:
        asset_data['SerialNumber'] = re.search(r'(Serial Number\s{1,}:\s{1,})(.{1,}?)(\s{1})',Nline_raw_text).group(2).strip()
    except AttributeError:
        pass
    
    try:# FMV/ACV FMV high low/single
        asset_data['FMV_Low'] = re.search(r'((Fair Market Value|(Actual Cash Value)) (of|in the)\s{1,}\$)((\d{1,}\,)?(\d{1,}))',Nline_raw_text).group(5).strip()
        asset_data['FMV_High'] = re.search(r'((Fair Market Value|(Actual Cash Value)) (of|in the)\s{1,}\$)((\d{1,}\,)?(\d{1,}))(\s*-\s*\$*)(\d{1,}\,\d{1,})',Nline_raw_text).group(9).strip()
    except AttributeError:
        print("x")
        pass
    
    try:# OLV/Auction value low/single
        print(re.search(r'(auction\s*\w* (of|in the)\s{1,}\$)(\d{1,}(\,\d{2,})?)((\s*.{1}\s*\$*)(\d{1,}(\,\d{2,})?))',Nline_raw_text))

        asset_data['OLV_Low'] = re.search(r'(auction\s*\w* (of|in the)\s{1,}\$)(\d{1,}(\,\d{2,})?)((\s*.{1}\s*\$*)(\d{1,}(\,\d{2,})?))',Nline_raw_text).group(3).strip()
        asset_data['OLV_High'] = re.search(r'(auction\s*\w* (of|in the)\s{1,}\$)(\d{1,}(\,\d{2,})?)((\s*.{1}\s*\$*)(\d{1,}(\,\d{2,})?))',Nline_raw_text).group(7).strip()
    except AttributeError:
        print("X")
        pass


FileNotFoundError: [Errno 2] No such file or directory: 'ADESK2020110002.pdf'

In [277]:
other_specs.update(Other_Specs0)
raw_specs=[]
with pdfplumber.open('ADECG2010120001.pdf') as pdf:   
    page = pdf.pages[1]
    raw_text = '\n' + page.extract_text()
    
    Nline_raw_text=re.sub('\•','\n•',raw_text)
    Nline_raw_text=re.sub('\n\n','\n',Nline_raw_text)

    raw_specs = re.findall(r'(?:\•\s{4,8})(.{1,})(?:\n)?',Nline_raw_text)

    index = 0
    for spec in raw_specs:
        #spec=re.sub(r"\&"," ",spec).strip()
        other_specs['Spec'+str(index+1)] = raw_specs[index].strip()
        index += 1

FileNotFoundError: [Errno 2] No such file or directory: 'ADECG2010120001.pdf'

In [278]:
other_specs

{'Spec1': '',
 'Spec2': '',
 'Spec3': '',
 'Spec4': '',
 'Spec5': '',
 'Spec6': '',
 'Spec7': '',
 'Spec8': '',
 'Spec9': '',
 'Spec10': '',
 'Spec11': '',
 'Spec12': '',
 'Spec13': '',
 'Spec14': '',
 'Spec15': '',
 'Spec16': '',
 'Spec17': '',
 'Spec18': '',
 'Spec19': '',
 'Spec20': '',
 'Spec21': '',
 'Spec22': '',
 'Spec23': '',
 'Spec24': '',
 'Spec25': '',
 'Spec26': '',
 'Spec27': '',
 'Spec28': '',
 'Spec29': '',
 'Spec30': '',
 'Spec31': '',
 'Spec32': '',
 'Spec33': '',
 'Spec34': '',
 'Spec35': '',
 'Spec36': '',
 'Spec37': '',
 'Spec38': '',
 'Spec39': ''}

In [279]:
Asset_Specs0={"Kms":"","Hours":"","Miles":"","Engine":"","EnginePower":"","Transmission":"","AxlesNo":"","GVWR":"","RearAxle":"",
              "FrontAxle":"","Sleeper":"","MooseBumper":""}
asset_specs.update(Asset_Specs0)
with pdfplumber.open("MNCCI2018060001.pdf") as pdf:
    page = pdf.pages[1]
    raw_text = '\n' + page.extract_text()

 
    try:
        asset_specs['Kms'] = re.search(r'(?:\•\s{4,8})(.*)(km|Km|kms|Kms|KMs|KM|kilometers|Kilometers)',raw_text).group(1).strip()
    except:
        pass
    
    try:
        asset_specs['EnginePower'] = re.search(r'(?:\•\s{4,8})(?:.*)(\d{3}\s{0,1})(?:(hp|HP|Hp))',raw_text).group(1)
    except:
        pass
    
    try:
        asset_specs['Miles'] = re.search(r'(?:\•\s{4,8})(.*)(miles|Miles|Mile|mile)',raw_text).group(1).strip()
    except:
        pass
    
    try:
        asset_specs['Hours'] = re.search(r'(?:\•\s{4,8})(.*)\s*(?:(hours|Hours|Hrs))',raw_text).group(1).strip()
    except:
        try:
            asset_specs['Hours'] = re.search(r'(?:\•\s{4,8})(.*)(?:(hours|Hours|Hrs))',raw_text).group(1).strip()
        except:
            pass    
       
    try:
        asset_specs['Engine'] = re.search(r'(?:\•\s{4,8})(.*engine|Engine)',raw_text).group(1)
    except:
        try:
            asset_specs['Engine'] = re.search(r'(?:\•\s{4,8})(.*diesel|Diesel)',raw_text).group(1)
        except:
            pass

    try:
        asset_specs['Transmission'] = re.search(r'(?:\•\s{4,8})(.*)(Transmission|transmission)',raw_text).group(1)
    except:
        pass
    
    try:
        asset_specs['AxlesNo'] = re.search(r'(Tandem|Tridem|(Single axles{0,1})|(Dual axles{0,1}))',raw_text).group(1)
    except:
        pass
    
    try:
        asset_specs['GVWR'] = re.search(r'(?:\•\s{4,8})(.*)(GVWR)',raw_text).group(1)
    except:
        pass
    
    try:
        asset_specs['RearAxle'] = re.search(r'(?:\•\s{4,8})(.*)(rear axles{0,1})',raw_text).group(1)
    except:
        pass

    try:
        asset_specs['FrontAxle'] = re.search(r'(?:\•\s{4,8})(.*)(front axle)',raw_text).group(1)
    except:
        pass

    try:
        asset_specs['Sleeper'] = re.search(r'(?:\•\s{4,8})(.*)(Sleeper|sleeper)',raw_text).group(1)
    except:
        pass
    
    try:
        asset_specs['MooseBumper'] = re.search(r'(?:\•\s{4,8})(.*)(Moose|Deer)',raw_text).group(1)
    except:
        pass
    



In [280]:
other_specs.update(Other_Specs0)
raw_specs=[]
with pdfplumber.open('MNCCI2018060001.pdf') as pdf:   
    page = pdf.pages[1]
    raw_text = '\n' + page.extract_text()
    
    Nline_raw_text=re.sub('\•','\n•',raw_text)
    Nline_raw_text=re.sub('\n\n','\n',Nline_raw_text)

    raw_specs = re.findall(r'(?:\•\s{4,8})(.{1,})(?:\n)?',Nline_raw_text)

    index = 0
    for spec in raw_specs:
        #spec=re.sub(r"\&"," ",spec).strip()
        other_specs['Spec'+str(index+1)] = raw_specs[index].strip()
        index += 1
other_specs

{'Spec1': '760,361 kilometers indicated (assumed rolled over)',
 'Spec2': 'Caterpillar C15 diesel engine (490hp)',
 'Spec3': '18 speed manual transmission',
 'Spec4': 'Tandem axle',
 'Spec5': '50,000 lbs GVWR',
 'Spec6': '12,000 lbs front axle',
 'Spec7': '38,000 lbs rear axles',
 'Spec8': '62" raised roof sleeper',
 'Spec9': 'Aerodynamic air deflector',
 'Spec10': 'Alloy wheels',
 'Spec11': 'Sliding fifth wheel hitch',
 'Spec12': 'Air ride suspension',
 'Spec13': 'External air cleaners',
 'Spec14': 'Dual chrome exhausts',
 'Spec15': 'Sun visor',
 'Spec16': 'Rear LED work lights',
 'Spec17': 'Cloth seats/ Full load',
 'Spec18': 'Remote lube system',
 'Spec19': '',
 'Spec20': '',
 'Spec21': '',
 'Spec22': '',
 'Spec23': '',
 'Spec24': '',
 'Spec25': '',
 'Spec26': '',
 'Spec27': '',
 'Spec28': '',
 'Spec29': '',
 'Spec30': '',
 'Spec31': '',
 'Spec32': '',
 'Spec33': '',
 'Spec34': '',
 'Spec35': '',
 'Spec36': '',
 'Spec37': '',
 'Spec38': '',
 'Spec39': ''}